# CNN Encoder
This notebook walks through a practical example of a Convolutional Neural Network (CNN) Encoder, used to compress an image into a latent representation. This representation is typically trained end-to-end with a CNN Decoder block, which aims to reconstruct the source image from this compressed representation. This steers the compressed representation into something that captures the key bits of information captured in the image. Once trained, this compressed representation can be used for downstream tasks such as image classification.

The CNN Encoder component typically consists of several convolutional operators stacked on top of each other. It is thus helpful to first gain an understanding of the convolutional operator. From a practical standpoint, it will be most helpful to understand the impact of the convolutional operator's parameters on the output shape of the image at each layer, as this will dictate the final vector size of the compressed representaiton

In what follows, we will have a practical look and dive into a CNN Encoder using PyTorch.

## Convolutional Operator
Let us first showcase a simple example with one convolutional operator.

First, we will create a simple tensor representing our source image. This image will consist 6 x 6 pixels and will have three channels representing Red, Green, and Blue (RGB). Note that Pytorch typically orders its tensors as $(C, H, W)$ channels, height, width.

In [10]:
import torch

source_image = torch.rand(3, 6, 6)
src_c, src_h, src_w = source_image.size()
print(f"Source image channels: {src_c}")
print(f"Source image height: {src_h}")
print(f"Source image width: {src_w}")


Source image channels: 3
Source image height: 6
Source image width: 6


It can help to visualise this:

![](images/src_image.png)

Now we can create our convolutional operator, with some intial parameters:

In [43]:
out_channels: int = 3
kernel_size: int = 2
stride: int = 1
padding: int = 0

conv = torch.nn.Conv2d(src_c, out_channels, kernel_size, stride, padding)

the `Conv2d` operator assumes that the first dimension of our image is the batch size, so let's quickly add an extra dimension to our sensor:

In [40]:
source_image_batched = source_image.unsqueeze(0)
source_image_batched.size()

torch.Size([1, 3, 6, 6])

Now we can pass our image through our convolution:

In [41]:
conv(source_image_batched)

tensor([[[[ 0.1243, -0.0383,  0.0540,  0.2570],
          [-0.0720, -0.0428, -0.1099,  0.1286],
          [-0.0322, -0.0347, -0.2259,  0.0105],
          [-0.0294,  0.1199,  0.0051, -0.2577]],

         [[-0.3270,  0.0657,  0.0728, -0.2501],
          [-0.2477, -0.1158, -0.1836,  0.0670],
          [-0.1083, -0.0005, -0.0597, -0.3975],
          [-0.1388, -0.4001, -0.1015, -0.3655]],

         [[-0.1409,  0.1122, -0.0472, -0.3194],
          [-0.1569,  0.1972, -0.2444, -0.0457],
          [ 0.0200, -0.0677,  0.0413, -0.0044],
          [-0.0496,  0.0391, -0.0649, -0.1058]]]],
       grad_fn=<ConvolutionBackward0>)

With our given parameters, we can see that the shape of our tensor has changed:

In [45]:
conv(source_image_batched).size()

torch.Size([1, 3, 5, 5])

Since we have not specified any padding, the two by two kernel, starting in the top left of the image, will slide along the image until it hits the edge. In this case, since there are 6 pixels in each row and column, the final operation will be on the (6 - 1)th pixel, as if it were to slide across once more, it would only have two pixels, rather than four, to operate on. 

This is much more easily seen in a diagram: